In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

In [11]:
# data = pd.read_csv("synthetic_network_traffic.csv")  # Replace with actual file
# scaler = MinMaxScaler()
# data_scaled = scaler.fit_transform(data[['packet_size']])  # Scale the 'packet_size' column

In [19]:
num_samples = 1000  # Total number of samples
sequence_length = 50  # Length of each sequence
packet_sizes = np.random.normal(loc=500, scale=100, size=num_samples)
connection_durations = np.random.normal(loc=2, scale=0.5, size=num_samples)

In [20]:
df = pd.DataFrame({
    'packet_size': packet_sizes,
    'connection_duration': connection_durations
})

# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)  # Scale the features

In [21]:
X = []
for i in range(num_samples - sequence_length):
    X.append(data_scaled[i:i + sequence_length])

X_train = np.array(X)  # Convert list to NumPy array

# Ensure the shape is (num_samples - sequence_length, sequence_length, 2)
print("Shape of training data:", X_train.shape)  # Should be (950, 50, 2)

Shape of training data: (950, 50, 2)


In [22]:
model = Sequential([
    LSTM(64, activation='relu', input_shape=(sequence_length, 2), return_sequences=True),
    LSTM(32, activation='relu', return_sequences=False),
    RepeatVector(sequence_length),
    LSTM(32, activation='relu', return_sequences=True),
    LSTM(64, activation='relu', return_sequences=True),
    TimeDistributed(Dense(2))
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 50, 64)            17152     
                                                                 
 lstm_21 (LSTM)              (None, 32)                12416     
                                                                 
 repeat_vector_5 (RepeatVec  (None, 50, 32)            0         
 tor)                                                            
                                                                 
 lstm_22 (LSTM)              (None, 50, 32)            8320      
                                                                 
 lstm_23 (LSTM)              (None, 50, 64)            24832     
                                                                 
 time_distributed_5 (TimeDi  (None, 50, 2)             130       
 stributed)                                           

In [23]:
history = model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1, shuffle=False)

Epoch 1/50
27/27 [==============================] - 8s 92ms/step - loss: 0.1200 - val_loss: 0.0551
Epoch 2/50
27/27 [==============================] - 2s 69ms/step - loss: 0.0422 - val_loss: 0.0407
Epoch 3/50
27/27 [==============================] - 2s 77ms/step - loss: 0.0366 - val_loss: 0.0372
Epoch 4/50
27/27 [==============================] - 2s 72ms/step - loss: 0.0334 - val_loss: 0.0342
Epoch 5/50
27/27 [==============================] - 2s 74ms/step - loss: 0.0315 - val_loss: 0.0327
Epoch 6/50
27/27 [==============================] - 2s 77ms/step - loss: 0.0303 - val_loss: 0.0315
Epoch 7/50
27/27 [==============================] - 2s 77ms/step - loss: 0.0294 - val_loss: 0.0306
Epoch 8/50
27/27 [==============================] - 2s 81ms/step - loss: 0.0286 - val_loss: 0.0299
Epoch 9/50
27/27 [==============================] - 2s 87ms/step - loss: 0.0280 - val_loss: 0.0294
Epoch 10/50
27/27 [==============================] - 2s 84ms/step - loss: 0.0275 - val_loss: 0.0290
Epoch 11/

In [18]:
# history = model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1, shuffle=False)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ANAN

In [24]:
model.save('model2.h5')

C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
